# Data analysis of Disney datasets - by Andrew Ostensen

# Introduction

## Question of interest
In this analysis, I will investigate the following question associated with this collection of Disney datasets.

**QUESTION:**  Which person (actor or director) has had the greatest financial impact on Disney revenues. This is interesting because Disney has different revenue streams and I would be interested to see which Disney animated movie(s) and therefore which actor/director is most impactful. I would expect the people associated with a blockbuster movie such as **The Lion King** to have had the most impact.

## Dataset description 

The datasets were obtained from the [website](https://data.world/kgarrett/disney-character-success-00-16) [1] and the following quote is from [website](https://en.wikipedia.org/wiki/List_of_Walt_Disney_Animation_Studios_films) [2].

"Walt Disney Animation Studios is an American animation studio headquartered in Burbank, California, the original feature film division of The Walt Disney Company. The studio's films are also often called "Disney Classics", or "Disney Animated Canon"

The Disney dataset is composed of $5$ tables, `disney_movies_total_gross.csv`, `disney_revenue_1991-2016.csv`, `disney-characters.csv`, `disney-director.csv` and `disney-voice-actors.csv` . Each table is stored in a `.csv` file and contains different information about Disney animated characters, movies, actors, directors and Disney corporate revenues. I will use the `disney_movies_total_gross.csv`,  `disney_revenue_1991-2016.csv`, `disney-director.csv`, and `disney-voice-actors.csv` tables formally described below:

* **disney_movies_total_gross.csv**
    * This file contains financial information on Disney movies (including non animated movies).  Columns include the movie title, the release date of the movie, the genre and MPAA rating, and its total gross and inflation adjusted gross take.
* **disney_revenue_1991_2016.csv**
    * This file contains financial information on Disney corporate revenue between 1991 and 2016, including the year, revenue from the Studio Entertainment division, revenue from the Consumer Products division, revenue from the Disney Interactive division, revenue from Parks & Resorts, and revenue from Disney Media Networks.
* **disney_director.csv**
    * This file contains information on the movie directors for Disney animated films, including the movie name and the name of the movie director.    
* **disney_voice_actors.csv**
    * This file includes information on the voice actors for Disney animated films, including the movie character name, the voice actor name and the movie name.

# Methods and Results

I am interested in analysing how the different people involved in making a movie can impact the revenue of the Disney corporation. For this analysis I will use the **disney_movies_total_gross**, the **disney_revenue_1991_2016**, the **disney_director** and the **disney_voice_actors** tables.

## Step 1
The first step is to import the tables and do some basic analysis such as observing null values and the data types for each dataframe.

In [ ]:
# Import all of the required libraries needed for this analysis
import altair as alt
import pandas as pd

# Import all of the required files
# Convert the 'release_date' column in the 'movies' DataFrame to a datetime64 datatype
directors = pd.read_csv('data/disney-director.csv')
actors = pd.read_csv('data/disney-voice-actors.csv')
movies = pd.read_csv('data/disney_movies_total_gross.csv', parse_dates=['release_date'])
revenue = pd.read_csv('data/disney_revenue_1991-2016.csv')

#### Step 1(a)
**directors** dataframe

In [ ]:
directors.head()

In [ ]:
directors.info()

The **directors** table has 56 rows and 2 columns. Each movie name has an associated director.  There are no missing entries and the datatype is appropriate for these columns.  Both the **name** and **director** columns are of interest for this analysis.

#### Step 1(b)
**actors** dataframe

In [ ]:
actors.head()

In [ ]:
actors.info()

In [ ]:
actors.describe()

The **actors** table has 935 rows and 3 columns. Each movie character has a corresponding voice actor and movie.  A voice actor can be associated with more than one movie character.  There are no missing entries and the datatype is appropriate for these columns.  

The most frequent voice-actor in the **actors** dataframe is **None**.  This is not useful for this analysis so therefore rows containing **None** should be deleted.

The **voice-actor** and **movie** columns are of interest for this analysis.

#### Step 1(c)
**movies** dataframe

In [ ]:
movies.head()

In [ ]:
movies.info()

The **movies** dataframe has 579 rows and 6 columns. Each movie has a corresponding release date, genre, MPAA rating, total gross and inflation adjusted gross.  The **genre** and **MPAA_rating** columns both have null values but are not of interest for this analysis.  

The **total_gross** and **inflation_adjusted_gross** columns have the wrong datatype - they need to be either int64 or float64.  The **movie_title**, **release_date** and **inflation_adjusted_gross** columns are of interest for this analysis.

#### Step 1(d)
**revenue** dataframe

In [ ]:
revenue.head()

In [ ]:
revenue.info()

The **revenue** dataframe has 26 rows and 7 columns. Each year has a corresponding revenue number from the following Disney divisions: 

        Studio Entertainment, 
        Disney Consumer Products, 
        Disney Interactive, 
        Walt Disney Parks and Resorts and 
        Disney Media Networks.      
A final column of the Total revenue per year is also provided.  Four of the columns have null data.  

The **Disney Media Networks** column has the wrong datatype - it needs to be float64.  

The **Year**, **Studio Entertainment** and **Disney Consumer Products** columns are of interest for this analysis.

## Step 2
The next step is to do some basic data wrangling such as fixing the improper data types and dealing with the null values observed in **Step 1**.

#### Step 2(a)
**movies** dataframe:

In [ ]:
# import the custom script
import string_conversion as sc

# Change the 'total_gross' and 'inflation_adjusted_gross' columns from str to int
movies = sc.string_conversion(movies, 'total_gross')
movies = sc.string_conversion(movies, 'inflation_adjusted_gross')

# Confirm the change is successful
movies.dtypes

In [ ]:
# Add a new 'Year' column to the dataframe
movies = movies.assign(Year=movies['release_date'].dt.year)

# Only keep the columns of interest
movies = movies[['movie_title', 'Year', 'inflation_adjusted_gross']]

# Confirm changes by viewing dataframe
movies.head()

Create a **movies_plt** dataframe to plot the top 10 grossing movies.

In [ ]:
# Sort the 'movies' dataframe by 'inflation_adjusted_gross'
movies_gross = (movies.sort_values(by='inflation_adjusted_gross', ascending=False)
                      .reset_index()
                      .loc[:9, ['movie_title', 'inflation_adjusted_gross']]
             )

# Plot the top 10 movies using a bar chart
top_10_movies_plt = (alt.Chart(movies_gross, width=500, height=300)
                        .mark_bar()
                        .encode(x=alt.X('movie_title:N', sort='-y', title='Movie'),
                                y=alt.Y('inflation_adjusted_gross:Q', title='Movie Gross ($)'))
                        .properties(title="Figure 1 - Top Grossing Disney Movies")
                    )
top_10_movies_plt

#### Step 2(b)
**revenue** dataframe:

First create a **discount** column to account for the effects of inflation.

Calculation is: [3]

            PV = FV/(1 + i)^n
            
            where: PV = present value
                   FV = future value
                   i  = inflation rate
                   n  = number of years
                   
 Inflation rate assumed to be 2.51%. [4]                   

In [ ]:
# Create 'discount' column
inflation = 0.0251
revenue = revenue.assign(Discount=(1.0/(1.0+inflation)**(revenue['Year']-revenue.loc[0, 'Year'])))

# Discount rates are relative to row 0 in the DataFrame (1991 dollars)
revenue[['Year', 'Discount']].head()

Create an **Entertainment** column from the existing **Studio Entertainment[NI 1]** column.

Modify as follows:

            Fill null values with the previous row's data.
            Create a rolling average with 2 years of data.
            Shift the data back one year.
            Apply the discount rate.

Example:  1995 Studio Entertainment revenue = average of 1995/1996 revenue (in 1991 dollars).

The assumption in this calculation is that a movie impacts revenue in both the current and subsequent year.

In [ ]:
# Create 'Entertainment' column
revenue = (revenue.assign(Entertainment=revenue['Studio Entertainment[NI 1]']
                  .fillna(method='bfill')
                  .rolling(window=2)
                  .mean()
                  .shift(-1))
           )
revenue = revenue.assign(Entertainment=revenue['Entertainment']*revenue['Discount'])
revenue[['Year', 'Entertainment']].head()

Create a **Consumer** column from the existing **Disney Consumer Products[NI 2]** column.

Modify using the same assumptions as the **Entertainment** column.

In [ ]:
# Create 'Consumer' column
revenue = (revenue.assign(Consumer=revenue['Disney Consumer Products[NI 2]']
                  .fillna(method='bfill')
                  .rolling(window=2)
                  .mean()
                  .shift(-1))
           )
revenue = revenue.assign(Consumer=revenue['Consumer']*revenue['Discount'])
revenue[['Year', 'Consumer']].head()

Create a **Total** column, which sums the **Entertainment** and **Consumer** columns and multiplies this value by 1000000 so that the value is in dollars.

The assumption in this calculation is that a movie impacts the Studio Entertainment and Consumer Products revenue but has minimal impact on Parks & Recreation, Media Networks & Interactive gaming revenue. 

Finally, drop any null values and only keep the required columns.

In [ ]:
# Create 'Total' column
revenue = (revenue.assign(Total=(revenue['Consumer']+revenue['Entertainment'])*1000000.0)
                  .reset_index()
           )

# Only keep the columns of interest
revenue = revenue[['Year', 'Consumer', 'Entertainment', 'Total']]

# Drop any null values
revenue = revenue.dropna()

# Confirm changes by viewing DataFrame
revenue

#### Step 2(c)
**actors** dataframe:

Remove the **None** actors from the dataframe.

In [ ]:
# Remove the 'None' actors
actors = actors[~(actors['voice-actor']=='None')]
actors.describe()

## Step 3
The next step is to combine the dataframes modified in **Step 2** to create a master dataframe for final visualization and analysis.

#### Step 3(a)
Merge **movies** and **revenue** dataframes.

In [ ]:
# Merge the 'movies' and 'revenue' dataframes
disney_df = movies.merge(revenue, how='inner', left_on='Year', right_on='Year')
disney_df.head()

#### Step 3(b)
Merge with the **actors** dataframe.

In [ ]:
# Merge the 'actors' dataframe
# Drop the 'character' column
disney_df = (disney_df.merge(actors, how='inner', left_on='movie_title', right_on='movie')
                                       .drop(columns=['character', 'movie'])
                        )
disney_df.head()

#### Step 3(c)
Merge with the **directors** dataframe.

In [ ]:
# Merge the 'directors' dataframe
disney_df = (disney_df.merge(directors, how='inner', left_on='movie_title', right_on='name')
                                       .drop(columns=['name'])
            )
disney_df.head()

#### Step 3(d)
Combine the **voice-actor** and **director** columns using the **melt()** method.

In [ ]:
# Melt the 'voice-actor' and 'director' columns into a single column called 'role'
disney_df = (disney_df.melt(id_vars=['Year', 'movie_title', 'inflation_adjusted_gross', 'Total'],
                         value_vars=['voice-actor', 'director'],
                         var_name=['role'],
                         value_name='name')
                     .drop_duplicates()
                     .reset_index(drop=True)
               )
disney_df.head()

## Step 4
The final step is to analyze the master **disney_df** dataframe to determine which person (actor or director) has had the largest amount of revenue attributed to them.

#### Step 4(a)
First, determine the total movie gross on a per year basis. 

In [ ]:
# Create a 'yr_movies' DataFrame 
yr_movies = (disney_df.groupby(by='Year')
                      .sum()['inflation_adjusted_gross']
                      .reset_index()
                      .rename(columns={'inflation_adjusted_gross': 'total_movies_gross'})
            )
yr_movies

#### Step 4(b)
Merge the **yr_movies** DataFrame with the **disney_df** DataFrame.

Create an **adjusted_revenue** column in the new DataFrame.

Calulation is:

            Fraction of a movies gross for a year / 
            Total Consumer Products & Entertainment revenue for a year   
The **adjusted_revenue** is assumed to be the movies contribution to corporate revenue for that year.

In [ ]:
# Merge the 'yr_movies' DataFrame
final_df = disney_df.merge(yr_movies, how='left', left_on='Year', right_on='Year')
final_df.head()

In [ ]:
# Create 'adjusted_revenue' column
final_df = final_df.assign(adjusted_revenue=final_df['inflation_adjusted_gross']/final_df['total_movies_gross']*final_df['Total'])
final_df.head()

#### Step 4(c)
Group the DataFrame by the actor/director **name** column.

Sum the **adjusted_revenue** column and then sort in a descending order.

This will show the person(s) with the highest total revenue contribution.

In [ ]:
names = (final_df.groupby(by='name')
               .sum()
               .sort_values(by='adjusted_revenue', ascending=False)
               .reset_index()
        )

# Keep the top 10 names for plotting
names_plt = names.loc[:9, ['name', 'adjusted_revenue']]
                            
names_plt

#### Step 4(d)
Next, plot the results to display each persons impact on adjusted revenue.

In [ ]:
# Visualize the top 10 people's contributions to revenue using a bar plot.
top_10_plot = (alt.Chart(names_plt, width=500, height=300).mark_bar()
                                                .encode(x=alt.X('name:N', sort='-y', title='Name'),
                                                        y=alt.Y('adjusted_revenue:Q', title='Adjusted Revenue ($)'))
                                                .properties(title="Figure 2 - Contributions to Revenue by Person")
         )
top_10_plot

#### Step 4(e)
Finally, view the movie(s) associated with each person in the previous step.

In [ ]:
# Merge the 'top_10_plot' dataframe with the 'disney_df' dataframe
# to link the highest contributors to their associated movies
name_movies = names_plt.merge(disney_df, how='inner', left_on='name', right_on='name')
name_movies = name_movies[['name', 'movie_title']]

name_movies

# Discussions

In this work, I analyzed the Disney datasets and tried to compute which person(s) had the most impact on company revenue. To complete this analysis I made several assumptions, the biggest assumption being that the fraction of a movies gross take for the year is directly correlated to the Disney corporation's studio entertainment and consumer products revenue.

It is quite surprising that **Don Hall** and **Daniel Henney** who worked on **Big Hero 6** are the biggest contributors. I would have guessed that someone who worked on a bigger hit movie such as **The Lion King** or someone with multiple movie credits such as third place contributor **David Ogden Stiers** would have been the biggest contributor to company revenue.

The biggest limitation to this analysis is that the datasets for actors and directors only include Disney Animation movies and do not include other Disney properties such as **Star Wars Ep. VII: The Force Awakens**. Consequently, if a non Disney Animation movies is a huge hit, the total movie gross will be larger for that year and the fraction attributed to the Disney Animation movie will be skewed.  Having additional data with actors and directors for all Disney movies would be an improvement.

# References

Refer to the following resources listed below which were used for this analysis.

## Resources used

* [Data Source](https://data.world/kgarrett/disney-character-success-00-16) [1]
    * The Disney datasets used in this work was curated by **Kelly Garrett**.
* [Quote Source](https://en.wikipedia.org/wiki/List_of_Walt_Disney_Animation_Studios_films) [2]
    * Information on Disney Studios was taken from **Wikipedia**.
* [Present Value Calculation](https://www.calculatorsoup.com/calculators/financial/present-value-calculator.php) [3]
    * The Present Value calculation is from **calculatorsoup.com**.
* [Inflation Rate](https://www.minneapolisfed.org/about-us/monetary-policy/inflation-calculator/consumer-price-index-1913-) [4]
    * The Inflation Rate is from the **Federal Reserve Bank of Minneapolis**.